# Data Import and Cleaning


This notebook describes the steps taken to import and clean source data.

#### Sources
- sat_2019_ca.csv: 2019 SAT scores for California by school
- act_2019_ca.csv: 2019 ACT scores for California by school
- ca_student_demogrs.txt: Enrollment demographics for California by school

In [1]:
# General imports
import pandas as pd
import numpy as np

#### 2019 SAT Scores for California by School
We'll start with the **2019 SAT scores** for California. Let's just see what we're working with.

In [2]:
# 2019 SAT scores for California by school
# Note: CDCode is an important ID field that needs to be read in as a string.
df_sat = pd.read_csv('../data/sat_2019_ca.csv',
                    dtype={'CDCode': str})
df_sat.info()
df_sat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CDS                    2579 non-null   float64
 1   CCode                  2579 non-null   float64
 2   CDCode                 2579 non-null   object 
 3   SCode                  2579 non-null   float64
 4   RType                  2579 non-null   object 
 5   SName                  1982 non-null   object 
 6   DName                  2521 non-null   object 
 7   CName                  2579 non-null   object 
 8   Enroll12               2579 non-null   float64
 9   NumTSTTakr12           2579 non-null   float64
 10  NumERWBenchmark12      2304 non-null   object 
 11  PctERWBenchmark12      2304 non-null   object 
 12  NumMathBenchmark12     2304 non-null   object 
 13  PctMathBenchmark12     2304 non-null   object 
 14  Enroll11               2579 non-null   float64
 15  NumT

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year,Unnamed: 25
0,6.615981e+12,6.0,0661598,630046.0,S,Colusa Alternative Home,Colusa Unified,Colusa,18.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
1,6.616061e+12,6.0,0661606,634758.0,S,Maxwell Sr High,Maxwell Unified,Colusa,29.0,10.0,...,*,*,*,*,*,*,*,*,2018-19,NaN
2,1.964733e+13,19.0,1964733,1930924.0,S,Belmont Senior High,Los Angeles Unified,Los Angeles,206.0,102.0,...,42,24.14,12,6.90,14,13.73,11,6.32,2018-19,NaN
3,1.964733e+13,19.0,1964733,1931476.0,S,Canoga Park Senior High,Los Angeles Unified,Los Angeles,227.0,113.0,...,97,35.27,37,13.45,18,15.93,35,12.73,2018-19,NaN
4,1.964733e+13,19.0,1964733,1931856.0,S,Whitman Continuation,Los Angeles Unified,Los Angeles,18.0,14.0,...,*,*,*,*,*,*,*,*,2018-19,NaN


The first thing I notice is **there's a lot of information that I don't need**.
<br>Working with this data will be much easier if I drop the columns that I know won't be necessary for analysis. I have a pretty good idea of what columns I _will need_ based on my problem statement. After double-checking the data dictionary, I decided to **drop some columns** right off the bat.
<br> I also **renamed the columns** to something a little more Pythonic.

In [3]:
df_sat = df_sat[['RType', 'CDCode', 'DName', 'Enroll12', 'NumTSTTakr12']]
df_sat.columns = ['r_type', 'district_id', 'district_name', 'count_students_12', 'count_sat_takers']
df_sat.head()

,r_type,district_id,district_name,count_students_12,count_sat_takers
0,S,0661598,Colusa Unified,18.0,0.0
1,S,0661606,Maxwell Unified,29.0,10.0
2,S,1964733,Los Angeles Unified,206.0,102.0
3,S,1964733,Los Angeles Unified,227.0,113.0
4,S,1964733,Los Angeles Unified,18.0,14.0


Great! That's easier on the eyes.
<br> The California Department of Education did a nice thing by providing an aggregate record for each school district. These are exactly the records I'm interested in because my problem statement is focused on _school districts_, not individual schools. School district records are indicated by a "D" in the `r_type` column.
<br> I'll **pull those records out with a Pandas query**, and overwrite the original dataframe (the _school_ and _state_ records are not necessary for analysis).

In [4]:
df_sat = df_sat.query('r_type == "D"')

At this point, I've isolated the records I care about from the source data. This is a good time to **check the data type** of each column, and to **check for missing values.**
<br>This is easily accomplised by calling `DataFrame.info()`

In [5]:
df_sat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539 entries, 2037 to 2575
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   r_type             539 non-null    object 
 1   district_id        539 non-null    object 
 2   district_name      539 non-null    object 
 3   count_students_12  539 non-null    float64
 4   count_sat_takers   539 non-null    float64
dtypes: float64(2), object(3)
memory usage: 25.3+ KB


I've narrowed things down to 539 records, each containing information about a unique school district...
<br>Before I get too confident, I'll confirm that we don't have any **duplicate records**.

In [6]:
df_sat['district_id'].duplicated().sum()  # Should equal zero if every record is unique.

0

Cool, no duplicates! That would've been a real pain.
<br>I did notice there are a few **districts with zero students** in the `count_students_12` field. This will cause problems in subsequent calculations, so I'd like to get rid of these records early, if possible.
<br> Let's see what we're dealing with:

In [7]:
df_sat.query('count_students_12 == 0')

,r_type,district_id,district_name,count_students_12,count_sat_takers
2051,D,2775150,Big Sur Unified,0.0,0.0
2538,D,3776901,SBE - Thrive Public,0.0,0.0
2561,D,1976992,SBE - Prepa Tec Los Angeles High,0.0,0.0


It appears we have three districts with zero grade-12 students and zero SAT-takers. These records comprise less than one percent of the total, and the missing information is probably not critical to the project. I will **drop these records** now.

In [8]:
df_sat.drop(df_sat.query('count_students_12 == 0').index, inplace=True)

Back to the problem statement - I'm interested in the **SAT participation rate** for each district.
<br>This metric isn't in the dataframe, but I can create a column and calculate it.

In [9]:
df_sat['sat_participation'] = df_sat.apply(lambda r: round(r['count_sat_takers'] / r['count_students_12'], 2), axis=1)
df_sat.head()

,r_type,district_id,district_name,count_students_12,count_sat_takers,sat_participation
2037,D,0161176,Fremont Unified,2537.0,845.0,0.33
2038,D,0161275,Piedmont City Unified,231.0,78.0,0.34
2039,D,0161291,San Leandro Unified,754.0,193.0,0.26
2040,D,1062166,Fresno Unified,4593.0,1048.0,0.23
2041,D,1075127,Mendota Unified,234.0,69.0,0.29


A couple more housekeeping things, and I'll be ready to move on from this data source... for now.

In [10]:
# Drop the r_type column. It's no longer necessary.
df_sat.drop('r_type', axis=1, inplace=True)

# Convert count_students_12 and count_sat_takers to integers.
# We're dealing with whole students here.
df_sat = df_sat.astype({'count_students_12': int,
                        'count_sat_takers': int})

# Reset the index
df_sat.reset_index(inplace=True, drop=True)

#### 2019 ACT Scores for California by School
Next we'll clean up the ACT scores for California. Below is a snapshot of what this data looks like:

In [11]:
# 2019 SAT scores for California by school
# Note: CDCode is an important ID field that needs to be read in as a string.
df_act = pd.read_csv('../data/act_2019_ca.csv',
                    dtype={'CDCode': str})
df_act.info()
df_act.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CDS          2309 non-null   float64
 1   CCode        2309 non-null   float64
 2   CDCode       2309 non-null   object 
 3   SCode        1787 non-null   float64
 4   RType        2309 non-null   object 
 5   SName        1729 non-null   object 
 6   DName        2251 non-null   object 
 7   CName        2309 non-null   object 
 8   Enroll12     2309 non-null   float64
 9   NumTstTakr   2309 non-null   float64
 10  AvgScrRead   1953 non-null   object 
 11  AvgScrEng    1953 non-null   object 
 12  AvgScrMath   1953 non-null   object 
 13  AvgScrSci    1953 non-null   object 
 14  NumGE21      1953 non-null   object 
 15  PctGE21      1953 non-null   object 
 16  Year         2309 non-null   object 
 17  Unnamed: 17  0 non-null      float64
dtypes: float64(6), object(12)
memory usage: 325.0+ K

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTstTakr,AvgScrRead,AvgScrEng,AvgScrMath,AvgScrSci,NumGE21,PctGE21,Year,Unnamed: 17
0,3.366993e+13,33.0,3366993,129882.0,S,21st Century Learning Institute,Beaumont Unified,Riverside,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
1,1.964212e+13,19.0,1964212,1995596.0,S,ABC Secondary (Alternative),ABC Unified,Los Angeles,58.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
2,1.563776e+13,15.0,1563776,1530377.0,S,Abraham Lincoln Alternative,Southern Kern Unified,Kern,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
3,4.369666e+13,43.0,4369666,4333795.0,S,Abraham Lincoln High,San Jose Unified,Santa Clara,463.0,53.0,23,22,22,23,34,64.15,2018-19,NaN
4,1.964733e+13,19.0,1964733,1935121.0,S,Abraham Lincoln Senior High,Los Angeles Unified,Los Angeles,226.0,19.0,21,20,23,22,11,57.89,2018-19,NaN


This looks **very similar** to the SAT data we just finished cleaning.
<br>I will not provide as much of a narrative for the steps that follow. They are basically the same steps that I took for the SAT data set.

In [12]:
# Keep only the columns necessary for analysis
# Change column names to snake case
df_act = df_act[['RType', 'CDCode', 'DName', 'Enroll12', 'NumTstTakr']]
df_act.columns = ['r_type', 'district_id', 'district_name', 'count_students_12', 'count_act_takers']

# Filter down to district records
df_act = df_act.query('r_type == "D"')

# Check for duplicate district_ids
if df_act['district_id'].duplicated().sum() != 0:
    raise Exception('df_act contains duplicate records!')

In [13]:
# The output of this query shows there are no districts in this dataset
# with zero grade-12 students enrolled.
df_act.query('count_students_12 == 0')

,r_type,district_id,district_name,count_students_12,count_act_takers


In [14]:
# Add a calculated column for act_participation
df_act['act_participation'] = df_act.apply(lambda r: round(r['count_act_takers'] / r['count_students_12'], 2), axis=1)

# And check our work
df_act.head()

,r_type,district_id,district_name,count_students_12,count_act_takers,act_participation
1122,D,0110017,Alameda County Office of Education,170.0,0.0,0.00
1123,D,0161119,Alameda Unified,919.0,155.0,0.17
1124,D,0161127,Albany City Unified,307.0,58.0,0.19
1125,D,0161143,Berkeley Unified,820.0,227.0,0.28
1126,D,0161150,Castro Valley Unified,737.0,106.0,0.14


In [15]:
# Checking to see if all values for act_participation fall between 0 and 1
(df_act['act_participation'].min(), df_act['act_participation'].max())

(0.0, 1.55)

In [16]:
# It appears there are some wild values > 1.
# Investigation time!

df_act.query('act_participation > 1')

,r_type,district_id,district_name,count_students_12,count_act_takers,act_participation
1400,D,3367157,Nuview Union,157.0,169.0,1.08
1504,D,4168999,Ravenswood City Elementary,20.0,31.0,1.55
1601,D,5271571,Los Molinos Unified,37.0,38.0,1.03


There are three records where `act_participation` is greater than one.
<br>This is because `count_act_takers` is greater than `count_students_12` in these districts.
<br>One possible explanation is that these districts had ACT takers who were _not_ HS seniors.
<br>Or it could be a reporting error.
<br><br>I'm going to **force these values down to 1**, just so we can keep `act_participation` within a standard range.
<br>These are such tiny school districts - they'll probably get dropped from the analysis anyway.

In [17]:
df_act.loc[df_act.query('act_participation > 1').index, 'act_participation'] = 1

Other housekeeping

In [18]:
# Drop the r_type column. It's no longer necessary.
df_act.drop('r_type', axis=1, inplace=True)

# Convert count_students_12 and count_sat_takers to integers.
# We're dealing with whole students here.
df_act = df_act.astype({'count_students_12': int,
                        'count_act_takers': int})

# Reset the index
df_act.reset_index(inplace=True, drop=True)

#### Enrollment Demographics for California by School
This dataset is quite different from the last two. Here's a first glance:

In [19]:
# This is a tab delimited file
# cds is an important id that needs to be read as a str
df_demogr = pd.read_csv('../data/2019_ca_enrollment.txt', sep='\t', dtype={'cds': str})
df_demogr.info()
df_demogr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120917 entries, 0 to 120916
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cds              120917 non-null  object 
 1   rtype            120917 non-null  object 
 2   schoolname       109778 non-null  object 
 3   districtname     120917 non-null  object 
 4   countyname       120904 non-null  object 
 5   studentgroup     120917 non-null  object 
 6   totalenrollment  120917 non-null  int64  
 7   subgrouptotal    120917 non-null  int64  
 8   rate             120917 non-null  float64
 9   reportingyear    120917 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


,cds,rtype,schoolname,districtname,countyname,studentgroup,totalenrollment,subgrouptotal,rate,reportingyear
0,00000000000000,X,State of California,State of California,NaN,AS,6186278,573925,9.3,2019
1,00000000000000,X,State of California,State of California,NaN,HOM,6186278,207677,3.4,2019
2,00000000000000,X,State of California,State of California,NaN,SWD,6186278,725412,11.7,2019
3,00000000000000,X,State of California,State of California,NaN,AA,6186278,334652,5.4,2019
4,00000000000000,X,State of California,State of California,NaN,HI,6186278,3374921,54.6,2019


First, I will **drop columns** that are unnecessary for analysis.
<br>I will also **rename columns** using snake case.

In [20]:
# df_demogr.drop(['schoolname', 'countyname', 'reportingyear'], axis=1, inplace=True)
df_demogr = df_demogr[['rtype', 'cds', 'districtname', 'studentgroup', 'totalenrollment', 'subgrouptotal', 'rate']]
df_demogr.columns=['r_type', 'cds_code', 'district_name', 'student_group', 'count_students', 'subgroup_total', 'rate']
df_demogr.head()

,r_type,cds_code,district_name,student_group,count_students,subgroup_total,rate
0,X,00000000000000,State of California,AS,6186278,573925,9.3
1,X,00000000000000,State of California,HOM,6186278,207677,3.4
2,X,00000000000000,State of California,SWD,6186278,725412,11.7
3,X,00000000000000,State of California,AA,6186278,334652,5.4
4,X,00000000000000,State of California,HI,6186278,3374921,54.6


I'm going to **create a new column** called `district_id`
<br>I'll use this column to perform joins with the other dataframes.
<br>From looking at the SAT and ACT data, I observed that the `district_id` is the first seven digits of the `cds_code`
<br>Pandas can extract this and store it in a new column.

In [21]:
df_demogr = df_demogr.assign(district_id=df_demogr['cds_code'].str[:7])

I'm going to **create a new dataframe** called `df_demogr_districts`
<br>This will eventually contain summary statistics on a per-district level.

In [22]:
df_demogr_districts = df_demogr.query('r_type == "D"')[['district_id', 'district_name', 'count_students']].drop_duplicates()

In [23]:
df_demogr_districts.head()

,district_id,district_name,count_students
13,0110017,Alameda County Office of Education,179
172,0161119,Alameda Unified,9383
426,0161127,Albany City Unified,3682
499,0161143,Berkeley Unified,9811
706,0161150,Castro Valley Unified,9269


I'm creating a column that shows the **number of schools** in each district.

In [24]:
df_demogr_districts = df_demogr_districts.merge(
    df_demogr.query('r_type == "S"').drop_duplicates('cds_code').groupby('district_id')['cds_code'].count(),
    how='left',
    left_on='district_id',
    right_index=True).rename(columns={'cds_code': 'count_schools'})

I discovered there's a district in here that doesn't have any schools listed.
<br>So, **I will drop it** (it's an elementary district, so not relavent to this project anyway)

In [25]:
df_demogr_districts[df_demogr_districts['count_schools'].isna()]

,district_id,district_name,count_students,count_schools
15677,1262927,Loleta Union Elementary,123,NaN


In [26]:
df_demogr_districts.drop(15677, axis=0, inplace=True)

Lastly, I want to make a column to show the **percentage of underrepresented minority (URM) students** in the district.
<br>The study defined URM groups as African American, Native American, and Hispanic/Latino.
<br> I will **add a column** that shows this percentage.

In [27]:
df_demogr_districts = df_demogr_districts.merge(
    df_demogr.query('r_type == "D" and student_group in ("AA", "AI", "HI")').groupby('district_id')['rate'].sum() / 100,
    how='left',
    left_on='district_id',
    right_index=True).rename(columns={'rate': 'pct_urm'})

# Check our work
df_demogr_districts.head()

,district_id,district_name,count_students,count_schools,pct_urm
13,0110017,Alameda County Office of Education,179,12.0,0.889
172,0161119,Alameda Unified,9383,21.0,0.225
426,0161127,Albany City Unified,3682,6.0,0.192
499,0161143,Berkeley Unified,9811,17.0,0.364
706,0161150,Castro Valley Unified,9269,15.0,0.293


This seems to have worked as intended. However, there are some NaN's in the `pct_urm` column.

In [28]:
df_demogr_districts[df_demogr_districts['pct_urm'].isna()]

,district_id,district_name,count_students,count_schools,pct_urm
15556,1262851,Green Point Elementary,19,1.0,NaN
15669,1262919,Kneeland Elementary,13,1.0,NaN
15687,1262935,Maple Creek Elementary,4,1.0,NaN
48018,2165375,Lincoln Elementary,4,1.0,NaN
114333,5371670,Coffee Creek Elementary,9,1.0,NaN
114391,5371761,Trinity Center Elementary,14,1.0,NaN


This occured because these school "districts" (note, they are only a single school) don't have any students from URM groups.
<br> We can safely replace these NaN's with zeros.

In [29]:
df_demogr_districts['pct_urm'].fillna(0, inplace=True)

In [30]:
df_demogr_districts.head()

,district_id,district_name,count_students,count_schools,pct_urm
13,0110017,Alameda County Office of Education,179,12.0,0.889
172,0161119,Alameda Unified,9383,21.0,0.225
426,0161127,Albany City Unified,3682,6.0,0.192
499,0161143,Berkeley Unified,9811,17.0,0.364
706,0161150,Castro Valley Unified,9269,15.0,0.293


Finally, just a little housekeeping to convert `count_schools` to an integer.

In [31]:
df_demogr_districts = df_demogr_districts.astype({'count_schools': int})

# Also reset the index
df_demogr_districts.reset_index(inplace=True, drop=True)

#### Merge and Export!

The goal here is to end up with **one dataframe** that we can export as a csv file. To do this, I'm going to merge the three dataframes that we just cleaned.

In [32]:
df_merged = df_sat.merge(df_act,
            how='outer',
            on='district_id',
            suffixes=('_sat', '_act')).merge(df_demogr_districts[['district_id','count_schools', 'count_students', 'pct_urm']],
                                            how='left',
                                            on='district_id')

Here's a look at what we end up with:

In [33]:
df_merged.isna().sum()

district_id               0
district_name_sat         0
count_students_12_sat     0
count_sat_takers          0
sat_participation         0
district_name_act        14
count_students_12_act    14
count_act_takers         14
act_participation        14
count_schools            14
count_students           14
pct_urm                  14
dtype: int64

In [34]:
df_merged.dropna(inplace=True)

In [35]:
df_merged.head()

,district_id,district_name_sat,count_students_12_sat,count_sat_takers,sat_participation,district_name_act,count_students_12_act,count_act_takers,act_participation,count_schools,count_students,pct_urm
0,0161176,Fremont Unified,2537,845,0.33,Fremont Unified,2537.0,511.0,0.20,42.0,35176.0,0.165
1,0161275,Piedmont City Unified,231,78,0.34,Piedmont City Unified,231.0,116.0,0.50,6.0,2596.0,0.100
2,0161291,San Leandro Unified,754,193,0.26,San Leandro Unified,754.0,78.0,0.10,12.0,8926.0,0.612
3,1062166,Fresno Unified,4593,1048,0.23,Fresno Unified,4593.0,574.0,0.12,105.0,70749.0,0.775
4,1075127,Mendota Unified,234,69,0.29,Mendota Unified,234.0,82.0,0.35,7.0,3543.0,0.974


In [36]:
df_merged = df_merged[['district_id', 'district_name_sat', 'count_schools', 'count_students',
                       'count_students_12_sat', 'sat_participation', 'act_participation', 'pct_urm']].rename(
    columns={'district_name_sat': 'district_name',
            'count_students_12_sat': 'count_students_12'})

In [37]:
df_merged = df_merged.astype({'count_schools': int,
                 'count_students': int})

df_merged['pct_urm'] = df_merged['pct_urm'].round(2)

df_merged.reset_index(inplace=True, drop=True)

In [38]:
df_merged.head()

,district_id,district_name,count_schools,count_students,count_students_12,sat_participation,act_participation,pct_urm
0,0161176,Fremont Unified,42,35176,2537,0.33,0.20,0.16
1,0161275,Piedmont City Unified,6,2596,231,0.34,0.50,0.10
2,0161291,San Leandro Unified,12,8926,754,0.26,0.10,0.61
3,1062166,Fresno Unified,105,70749,4593,0.23,0.12,0.78
4,1075127,Mendota Unified,7,3543,234,0.29,0.35,0.97


In [40]:
df_merged.to_csv('../data/cleaned_data.csv', index=False)